In [1]:
#pip install pulp
from pulp import *
import random

In [2]:
m = 5 #number of potential facility locations
n = 6 #number of demand points

#k=2 #If there are a Fixed total number of facilities to be established.

#Costs of establishing facilities (In potential locations)
f = [12, 5, 3, 7, 9]

#Costs of serving demand points. cij: Transportation cost, town i to facility j
c = [[2, 3, 6, 7, 1],
[0, 5, 8, 4, 12],
[11, 6, 14, 5, 8],
[19, 18, 21, 16, 13],
[3, 9, 8, 7, 10],
[4, 7, 9, 6, 0]]
            

#ID to towns (0 to n-1) and Facilities (0 to m-1)
Town = list(range(0, n)) #number of towns
Facility = list(range(0, m)) #number of facilities


In [3]:
#Defining Decision Variables

#Define decision variable yj.

y = LpVariable.dicts("y", Facility, 0, 1, LpBinary) #0 or 1, binary variable.

#Define decision variable xij.
x = LpVariable.dicts("x", [(i,j) for i in Town for j in Facility], 0, 1) 
#Assume xij is a continuous variable, between 0 and 1. So haven't mentioned LpBinary.



In [4]:
#Problem
flp = LpProblem("LP_Problem", LpMinimize)

# Decision variables:
# 0 <= xij <=1 (If the ith town is serviced by the jth facility)
# For each i: sum(xij) = 1.
# yj: If jth facility is opened or not. (yj is 0 or 1)
# Constraint: sum(yj) <= k (If take k)
# Additional constraint: If yj = 0, xij = 0 --> xij <= yj


#Set up the objective function in flp

flp += lpSum(f[j]*y[j] for j in Facility)+lpSum(x[(i, j)]*c[i][j] for i in Town for j in Facility)

#Constraints: 
# 1. Sum(xij) = 1
# 2. Sum(yj) = k (k facities opened) :: If taking k.
# 3. If (yj=0), all xij=0 ---> xij<=yj.

#1
for i in Town:
    flp += lpSum(x[(i, j)] for j in Facility) == 1


#2 :: If k is being involved
# for j in Facility:
#     flp += lpSum(y[j] for j in Facility) == k

#3
for i in Town:
    for j in Facility:
        flp += x[(i,j)] <= y[j]




In [5]:
flp.solve()


Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/pranavv3/opt/anaconda3/lib/python3.9/site-packages/pulp/apis/../solverdir/cbc/osx/64/cbc /var/folders/hg/0pz349td64l99xls4nwlkwc00000gn/T/33acaee387e348ea9cf07d75004ad88c-pulp.mps timeMode elapsed branch printingOptions all solution /var/folders/hg/0pz349td64l99xls4nwlkwc00000gn/T/33acaee387e348ea9cf07d75004ad88c-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 41 COLUMNS
At line 175 RHS
At line 212 BOUNDS
At line 248 ENDATA
Problem MODEL has 36 rows, 35 columns and 90 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 45.5 - 0.00 seconds
Cgl0004I processed model has 36 rows, 35 columns (5 integer (5 of which binary)) and 90 elements
Cbc0038I Initial state - 3 integers unsatisfied sum - 1.5
Cbc0038I Pass   1: suminf.    0.00000 (0) obj. 51 iterations 9
Cbc0038I Solution found of 51
Cbc00

1

In [6]:
print("Solution Status = ", LpStatus[flp.status])


Solution Status =  Optimal


In [7]:
# Print Value of the Objective Function in the optimal case
print("Total Cost = ", value(flp.objective))


Total Cost =  46.0


In [8]:
#If x(i, j)=1, Town i is allocated Facility j. NOTE:: i, j are 0-indexed in this case
for v in flp.variables():
    print(v.name, "=", v.varValue)


x_(0,_0) = 0.0
x_(0,_1) = 0.0
x_(0,_2) = 0.0
x_(0,_3) = 0.0
x_(0,_4) = 1.0
x_(1,_0) = 0.0
x_(1,_1) = 0.0
x_(1,_2) = 0.0
x_(1,_3) = 1.0
x_(1,_4) = 0.0
x_(2,_0) = 0.0
x_(2,_1) = 0.0
x_(2,_2) = 0.0
x_(2,_3) = 1.0
x_(2,_4) = 0.0
x_(3,_0) = 0.0
x_(3,_1) = 0.0
x_(3,_2) = 0.0
x_(3,_3) = 0.0
x_(3,_4) = 1.0
x_(4,_0) = 0.0
x_(4,_1) = 0.0
x_(4,_2) = 0.0
x_(4,_3) = 1.0
x_(4,_4) = 0.0
x_(5,_0) = 0.0
x_(5,_1) = 0.0
x_(5,_2) = 0.0
x_(5,_3) = 0.0
x_(5,_4) = 1.0
y_0 = 0.0
y_1 = 0.0
y_2 = 0.0
y_3 = 1.0
y_4 = 1.0


In [9]:
# Print the solution of Binary Decision Variables
Tolerance = 0.0001
for j in Facility:
    if y[j].varValue > Tolerance:
        print("Estalish Facility at site = ", j)


Estalish Facility at site =  3
Estalish Facility at site =  4
